
**Connect to Elasticsearch**

In [2]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

client_info = es.info()

print("Connected to Elsticsearch!")

pprint(client_info.body)

Connected to Elsticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '64c49e436740',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


**Create Index**

In [3]:
es.indices.delete(index="my_index", ignore_unavailable=True) 
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

**Indexing Documents**

In [4]:
import json 
from tqdm import tqdm

documents = json.load(open("data/astronomy.json"))

operations = []

for document in tqdm(documents, total=len(documents)):
    operations.append({"index": {"_index": "my_index"}}) # action
    operations.append(document)

# bulk api
response = es.bulk(operations=operations)

pprint(response.body)

100%|██████████| 10/10 [00:00<00:00, 57932.38it/s]

{'errors': False,
 'items': [{'index': {'_id': 'hLOuW5oBmU94NI-MrmJD',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'hbOuW5oBmU94NI-MrmJD',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 1,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'hrOuW5oBmU94NI-MrmJD',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 2,
                      '_shards': {'failed': 0, 'successful': 1, '

In [5]:
response = es.count(index="my_index")
count = response.body["count"]
print(f"Total documents found in my_index: {count}")

Total documents found in my_index: 0


**SQL search API**
# 
1. Simple select query

In [6]:
query = {
    "query": "SELECT title FROM my_index ORDER BY id LIMIT 5"
}

result = es.sql.query(body=query)

for row in result['rows']:
    print(row)

['The Solar System']
['Black Holes']
['Galaxies']
['The Big Bang Theory']
['Exoplanets']


2. Change response format

In [7]:
query = {
    "query": "SELECT * FROM my_index"
}

result = es.sql.query(body=query, format='json')

print(result)

{'columns': [{'name': 'content', 'type': 'text'}, {'name': 'id', 'type': 'long'}, {'name': 'title', 'type': 'text'}], 'rows': [['The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.', 1, 'The Solar System'], ['A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.', 2, 'Black Holes'], ['Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.', 3, 'Galaxies'], ['The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.', 4, 'The Big Bang Theory'], ['Exoplanets, or extrasolar planets, are plane

In [8]:
query = {
    "query": "SELECT * FROM my_index"
}

result = es.sql.query(body=query, format='txt')

print(result)

                                                                                                        content                                                                                                         |      id       |         title         
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------------
The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.                                          |1              |The Solar System       
A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.                      |2           

In [9]:
query = {
    "query": "SELECT * FROM my_index"
}

result = es.sql.query(body=query, format='csv')

print(result)

content,id,title
"The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.",1,The Solar System
"A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.",2,Black Holes
"Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.",3,Galaxies
The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.,4,The Big Bang Theory
"Exoplanets, or extrasolar planets, are planets that exist outside our solar system. They vary greatly in size and composition and are often found using methods like the transit method a

3. Filter results

In [10]:
query = {
    "query": "SELECT * FROM my_index",
    "filter": {
        "term": {
            "title.keyword": "Black Holes"
        }
    }
}

result = es.sql.query(body=query)
response.body

{'count': 0,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

4. Pagination

In [11]:
query = {
    "query": "SELECT content, id, title FROM my_index",
    "fetch_size": 2  # fetch up to 100 rows per batch
}

# Execute SQL query
result = es.sql.query(body=query, format="json")  # format=json for easy parsing
pprint(result.body) 

{'columns': [{'name': 'content', 'type': 'text'},
             {'name': 'id', 'type': 'long'},
             {'name': 'title', 'type': 'text'}],
 'cursor': '6s2wBERGTAB0j01OwzAQhec5Qa26YZdzpGIDyyAkBJFSQIUib6KqsYnBTUQZ6M+SE7HgGlyklyhjCN3xFtaM3rynz9CEmhRIKRLtRIdhEClgYJ3xVfnSLjguq3a2Q+cR4RPRTwgSDm/nyILQgqg3axs2DcuVq0gdsGNvpHWz/fqI0Z+vS9dUZpUMj43lbDgfLycbX/uRmdq0WBKSk3N79pyuL8favL2mWuvHe5dn1DHgPbnNisnRRXl17Ub53YLrp5siPw3JfyqJBvtPRrB/gIjZrFi4rYAi9m3zEJZf4L3ZI/oGAAD//wMA',
 'rows': [['The Solar System consists of the Sun and the objects that orbit '
           'it, including eight planets, their moons, dwarf planets, and '
           'countless small bodies like asteroids and comets.',
           1,
           'The Solar System'],
          ['A black hole is a region of space where the gravitational pull is '
           'so strong that nothing, not even light, can escape from it. They '
           'are formed when massive stars collapse under their own gravity.',
           2,
           'Bla

In [12]:
while "cursor" in result.body:
    result = es.sql.query(
        cursor=result.body["cursor"],
        format='json'
    )

    pprint(result.body)
    print()

{'cursor': '6s2wBERGTAB0jz1uwkAQhWfWjkA0dD6HUZqkdBQJgSVDIhKibSyEd+OFxRYw/JaciIJrcJFcAmYdQpdXzd97+gYlYA4CQQhgXVhNV7AEYkMbZbN0WS7IT7NyfMHbDgBP6FUmZDOAV9ldwROXgl5tXBakCuIrk4F4IENWcer+53z0sT7bpabI1DZoPSlNUWs22Az3Nrc9NdJhsgEMntv6dR7uugOp1qtQSjn5MnEENwY8BB9RMnzspP0304s/F5RP35P4xTn/iQRo3J/0UP8Bok9qS8ytGRR9WxbfrvkFvi9rAFcAAAD//wMA',
 'rows': [['Galaxies are vast systems that consist of stars, stellar remnants, '
           'interstellar gas, dust, and dark matter. The Milky Way is the '
           'galaxy that contains our Solar System.',
           3,
           'Galaxies'],
          ['The Big Bang Theory is the leading explanation about how the '
           'universe began. It suggests that the universe was once in an '
           'extremely hot and dense state and has been expanding ever since.',
           4,
           'The Big Bang Theory']]}

{'cursor': '6s2wBERGTAB0jz1uwkAQhWfWRiCadD6HURoojZBQYslARABtYyG8ixcWWyHDb5kTpcg1chEuQWaB0PGq+XtP36AEzEEgCAGsM+vJFSyBWNdG2Sz9LNfkp1k5O+NtB4A/6F1My

5. Trasnlate API

In [20]:
translate_query = {
    "query": "SELECT * FROM my_index WHERE content LIKE '%universe%' ORDER BY id DESC LIMIT 5"
}

result = es.sql.translate(
    body=translate_query
)

pprint(result.body)

{'_source': False,
 'fields': [{'field': 'content'}, {'field': 'id'}, {'field': 'title'}],
 'query': {'wildcard': {'content.keyword': {'boost': 1.0,
                                            'wildcard': '*universe*'}}},
 'size': 5,
 'sort': [{'id': {'missing': '_first',
                  'order': 'desc',
                  'unmapped_type': 'long'}}],
 'track_total_hits': -1}


### SQL Translate API

The Elasticsearch SQL translate API is used to convert a SQL query into its equivalent Elasticsearch internal query, known as Query DSL (Domain-Specific Language). In real life, it is a crucial tool primarily used by developers for: 
Learning and Debugging: The primary real-life use is for developers to learn how specific SQL queries are executed internally by Elasticsearch. By seeing the generated Query DSL, they can understand the underlying logic and structure of how data is queried in Elasticsearch, which acts as both a good teacher and a dictionary for the DSL.
#
**Query Optimization**: Developers can use the translated Query DSL to inspect and ensure that their SQL queries are running optimally. This helps in identifying potential performance bottlenecks and understanding how the query leverages Elasticsearch's capabilities like relevance scoring, aggregations, and full-text search.
#
**Transitioning to Query DSL**: It facilitates a smooth transition for developers or data scientists who are familiar with traditional SQL but eventually need to work with the more powerful and flexible JSON-based Query DSL for complex or performance-critical tasks.
#
**Building Applications with Native Clients**: In scenarios where developers might be using a low-level client or need to incorporate the generated DSL into existing code that relies on the Query DSL format, the translate API provides a programmatic way to get the correct JSON query structure without manually writing it.
#
**Integration with BI Tools (Indirectly)**: While BI tools generally use the standard SQL search API with JDBC/ODBC drivers, the translate API helps the developers who build or maintain these integrations to understand and debug how the BI tool's SQL requests are being handled by Elasticsearch. 
#
Essentially, it serves as a powerful educational and debugging aid, bridging the gap between the familiar SQL syntax and the native Elasticsearch Query DSL. 